In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import glob
import os
import joblib

In [3]:
# Grab all csvs
csvs = glob.glob(os.path.join('./data', '*.csv'))

# Loop through files, convert to dfs, tidy headers and store in a list
dataframes = [] 
#Actual data starts at row 6 and some of the columns have np.nan as header and all values.
for csv in csvs:
    df = pd.read_csv(csv, header=None)
    df.columns = df.iloc[5].values
    df = df[6:].reset_index(drop=True).loc[:, df.columns.notna()]
    dataframes.append(df)

In [4]:
csvs

['./data/Anxiety-Table 1.csv',
 './data/Drive to employer-Table 1.csv',
 './data/Broadband availability-Table 1.csv',
 './data/Worthwhile-Table 1.csv',
 './data/Cycle to employer-Table 1.csv',
 './data/Male HLE-Table 1.csv',
 './data/Good or outstanding schools-Table 1.csv',
 './data/Notes-Table 1.csv',
 './data/Employment rate-Table 1.csv',
 './data/Data inclusivity-Table 1.csv',
 './data/Public transport to employer-Table 1.csv',
 './data/New houses-Table 1.csv',
 './data/GCSE-Table 1.csv',
 './data/Female HLE-Table 1.csv',
 './data/Weekly pay-Table 1.csv',
 './data/GVA per hour-Table 1.csv',
 './data/4G-Table 1.csv',
 './data/Overweight adults-Table 1.csv',
 './data/Overweight children-Table 1.csv',
 './data/Early yrs lit-Table 1.csv',
 './data/Data dictionary-Table 1.csv',
 './data/Smokers-Table 1.csv',
 './data/Life satisfaction-Table 1.csv',
 './data/Apprenticeships completions-Table 1.csv',
 './data/Early yrs comms-Table 1.csv',
 './data/Apprenticeships starts-Table 1.csv',
 './

In [5]:
# Merge dfs into one big df
merged = dataframes[0]
for i, frame in enumerate(dataframes[1:]):
    #Try/Except because some of the csvs contain notes rather than data
    try:
        merged = pd.merge(merged, frame, how='left', on=['Area Code', 'Area'])
    except:
        continue

In [6]:
#Remove columns with notes or nan in the title
for col in list(merged.columns):
    try:
        if ('Note' in col) or ('nan' in col):
            merged.drop(columns=col, inplace=True)
        else:
            continue
    except:
        continue

In [7]:
merged.columns

Index(['Area Code', 'Area',
       'Mean anxiety yesterday scored 0 (not at all) - 10 (completely)',
       'Average travel time in minutes_x',
       'Premises with gigabit capable broadband %',
       'Mean feeling things done in life are worthwhile scored 0 (not at all) - 10 (completely)',
       'Average travel time in minutes_y', 'Male healthy life expectancy',
       'HLE Lower CI_x', 'HLE Upper CI_x',
       'Schools and nurseries rated good or outstanding by OFSTED %',
       'Employment rate %', 'Average travel time in minutes',
       'Absolute increase in dwelling stock (including new builds, conversions, changes of use, other gains/losses and offset by demolitions)',
       'Young people achieving GCSEs (and equivalent) in English and Maths  %',
       'Female healthy life expectancy', 'HLE Lower CI_y', 'HLE Upper CI_y',
       'Median weekly pay (£)', 'Gross Value Added per hour (£)',
       'Geographic areas with 4G signal outdoors from at least 1 operator %',
       'Ove

In [8]:
#Rename columns
merged.columns = ['area_code', 
                  'area',
                  'anxiety',
                  'car_avg_travel',
                  'broadband',
                  'worthwhile',
                  'cycle_avg_travel', 
                  'male_hle',
                  'male_hle_lower_ci', 
                  'male_hle_upper_ci',
                  'schools_nurseries',
                  'employment_rate', 
                  'public_transport_avg_travel',
                  'increase_dwellings',
                  'gcses',
                  'female_hle', 
                  'female_hle_lower_ci', 
                  'female_hle_upper_ci',
                  'income', 
                  'gva_ph',
                  '4g',
                  'overweight_adults', 
                  'overweight_adults_lower_ci_95',
                  'overweight_adults_upper_ci_95', 
                  'overweight_adults_lower_ci_99.8',
                  'overweight_adults_upper_ci_99.8', 
                  'overweight_children', 
                  'overweight_children_lower_ci_95',
                  'overweight_children_upper_ci_95', 
                  'overweight_children_lower_ci_99.8',
                  'overweight_children_upper_ci_99.8', 
                  'early_yrs_lit',
                  'smoking', 
                  'smoking_lower_ci', 
                  'smoking_upper_ci',
                  'life_satisfaction',
                  'apprenticeships_completions',
                  'early_yrs_comms',
                  'apprenticeships_starts',
                  'early_yrs_maths',
                  'happiness']

In [9]:
merged.head()

,area_code,area,anxiety,car_avg_travel,broadband,worthwhile,cycle_avg_travel,male_hle,male_hle_lower_ci,male_hle_upper_ci,...,early_yrs_lit,smoking,smoking_lower_ci,smoking_upper_ci,life_satisfaction,apprenticeships_completions,early_yrs_comms,apprenticeships_starts,early_yrs_maths,happiness
0,E06000001,Hartlepool,3.39,7.8,5.3,7.68,10.1,57.0,55.3,58.8,...,73.5,15.9,12.4,19.4,7.33,370,79.3,660,76.9,7.21
1,E06000002,Middlesbrough,3.11,8.5,5.6,7.62,11.3,58.5,56.8,60.2,...,63.9,12.8,9.2,16.3,7.21,470,71.8,900,68.2,7.26
2,E06000003,Redcar and Cleveland,3.16,8.1,1.5,7.76,11.1,60.2,58.4,62.1,...,72.2,13.4,10.6,16.3,7.44,560,79.9,980,77.6,7.49
3,E06000004,Stockton-on-Tees,3.25,7.8,5.7,7.74,10.1,57.8,55.8,59.7,...,74.6,10.8,7.8,13.8,7.40,740,84.4,1400,79.9,7.25
4,E06000005,Darlington,3.63,7.5,2.9,7.55,9.5,58.1,56.2,60.0,...,73.2,13.5,10.0,16.9,7.25,390,78.1,730,77.1,7.12


In [10]:
#Check dtypes
merged.dtypes

area_code                            object
area                                 object
anxiety                              object
car_avg_travel                       object
broadband                            object
worthwhile                           object
cycle_avg_travel                     object
male_hle                             object
male_hle_lower_ci                    object
male_hle_upper_ci                    object
schools_nurseries                    object
employment_rate                      object
public_transport_avg_travel          object
increase_dwellings                   object
gcses                                object
female_hle                           object
female_hle_lower_ci                  object
female_hle_upper_ci                  object
income                               object
gva_ph                               object
4g                                   object
overweight_adults                    object
overweight_adults_lower_ci_95   

All data types are strings but are actually given as numerical values, so we need to change them to floats.

In [11]:
#Check number of nan before conversion
merged.isna().sum().sum()

3525

In [12]:
merged.isna().sum()

area_code                              0
area                                   0
anxiety                                0
car_avg_travel                        76
broadband                             11
worthwhile                             0
cycle_avg_travel                      76
male_hle                             192
male_hle_lower_ci                    192
male_hle_upper_ci                    192
schools_nurseries                    258
employment_rate                       11
public_transport_avg_travel           76
increase_dwellings                    69
gcses                                 65
female_hle                           192
female_hle_lower_ci                  192
female_hle_upper_ci                  192
income                                11
gva_ph                                22
4g                                    11
overweight_adults                     76
overweight_adults_lower_ci_95         76
overweight_adults_upper_ci_95         76
overweight_adult

In [13]:
#list of column names
cols = list(merged.columns[2:])

merged_flt = merged.copy()

#Convert all dtypes to float
merged_flt[cols] = merged[cols].apply(pd.to_numeric, errors='coerce')

In [14]:
#Check conversion
merged_flt.dtypes

area_code                             object
area                                  object
anxiety                              float64
car_avg_travel                       float64
broadband                            float64
worthwhile                           float64
cycle_avg_travel                     float64
male_hle                             float64
male_hle_lower_ci                    float64
male_hle_upper_ci                    float64
schools_nurseries                    float64
employment_rate                      float64
public_transport_avg_travel          float64
increase_dwellings                   float64
gcses                                float64
female_hle                           float64
female_hle_lower_ci                  float64
female_hle_upper_ci                  float64
income                               float64
gva_ph                               float64
4g                                   float64
overweight_adults                    float64
overweight

In [15]:
(merged_flt.isna().sum() - merged.isna().sum())

area_code                             0
area                                  0
anxiety                               9
car_avg_travel                        0
broadband                             0
worthwhile                            5
cycle_avg_travel                      0
male_hle                              2
male_hle_lower_ci                     2
male_hle_upper_ci                     2
schools_nurseries                     0
employment_rate                       2
public_transport_avg_travel           0
increase_dwellings                    0
gcses                                 1
female_hle                            2
female_hle_lower_ci                   2
female_hle_upper_ci                   2
income                                2
gva_ph                                0
4g                                    0
overweight_adults                     0
overweight_adults_lower_ci_95         0
overweight_adults_upper_ci_95         0
overweight_adults_lower_ci_99.8       0


In [16]:
#159 additional nan found
merged_flt.isna().sum().sum() - merged.isna().sum().sum()

159

In [17]:
joblib.dump(merged_flt, 'full_df.jlib')

['full_df.jlib']